In [ ]:
from pathlib import Path
from typing import cast

import numpy as np
import plotly.graph_objects as go
import polars as pl

In [167]:
def get_data(source: str | Path, sheet_name: str) -> pl.DataFrame:
    return cast(
        pl.DataFrame,
        pl.read_excel(source=source, sheet_name=sheet_name),
    )

In [168]:
def calculate_spot_curve(df: pl.DataFrame) -> pl.DataFrame:
    payment_dates = df["maturity date"].unique().sort().to_list()

    df_indexed = df.with_row_index("bond_idx")

    cashflows = (
        df_indexed.select(
            [
                "bond_idx",
                "KYTREASNO",
                "maturity date",
                (pl.col("cpn rate") / 2).alias("coupon"),
            ]
        )
        .join(
            pl.DataFrame({"maturity date": payment_dates}).with_row_index("date_idx"),
            on="maturity date",
        )
        .with_columns([pl.lit(payment_dates).alias("all_dates")])
        .explode("all_dates")
        .with_columns(
            [pl.col("all_dates").rank("dense").over("bond_idx").alias("payment_idx")]
        )
        .with_columns(
            [
                pl.when(pl.col("payment_idx") <= pl.col("date_idx") + 1)
                .then(pl.col("coupon"))
                .otherwise(0)
                .alias("cashflow")
            ]
        )
        .with_columns(
            [
                pl.when(pl.col("all_dates") == pl.col("maturity date"))
                .then(pl.col("cashflow") + 100)
                .otherwise(pl.col("cashflow"))
                .alias("cashflow")
            ]
        )
        .pivot(index="bond_idx", on="all_dates", values="cashflow")
        .sort("bond_idx")
    )

    C = cashflows.select(pl.all().exclude("bond_idx")).to_numpy()
    price_vec = df_indexed.sort("bond_idx").select("price").to_numpy().flatten()

    z = np.linalg.solve(C, price_vec)

    result = df.with_columns(
        [
            pl.Series("discount_factor", z),
        ]
    ).with_columns(
        [
            (
                2 * ((1 / pl.col("discount_factor")) ** (1 / (2 * pl.col("ttm"))) - 1)
            ).alias("r_semi"),
            (-pl.col("discount_factor").log() / pl.col("ttm")).alias("r_cont"),
        ]
    )

    return result

In [169]:
def calculate_forward_curve(spot_curve: pl.DataFrame) -> pl.DataFrame:
    forwards = (
        spot_curve.sort("ttm")
        .with_columns(
            [
                pl.col("ttm").alias("T_start"),
                pl.col("ttm").shift(-1).alias("T_end"),
                pl.col("discount_factor").alias("Z_start"),
                pl.col("discount_factor").shift(-1).alias("Z_end"),
            ]
        )
        .filter(pl.col("T_end").is_not_null())
        .with_columns(
            [
                (pl.col("Z_end") / pl.col("Z_start")).alias("forward_discount"),
                (pl.col("T_end") - pl.col("T_start")).alias("delta_T"),
            ]
        )
        .with_columns(
            [
                (
                    2
                    * (
                        (1 / pl.col("forward_discount"))
                        ** (1 / (2 * pl.col("delta_T")))
                        - 1
                    )
                ).alias("forward_rate_semi"),
                (-pl.col("forward_discount").log() / pl.col("delta_T")).alias(
                    "forward_rate_cont"
                ),
            ]
        )
        .select(
            [
                "T_start",
                "T_end",
                "forward_discount",
                "forward_rate_semi",
                "forward_rate_cont",
            ]
        )
    )

    return forwards

In [170]:
def plot_spot_curve(spot_curve: pl.DataFrame):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=spot_curve["ttm"],
            y=spot_curve["discount_factor"],
            mode="lines+markers",
            name="Discount Factor Z(T)",
            line=dict(color="#8B5CF6", width=2.5),
            marker=dict(size=6, color="#8B5CF6"),
            hovertemplate="T: %{x:.4f}<br>Z(T): %{y:.4f}<extra></extra>",
        )
    )

    fig.update_layout(
        title=dict(
            text="Spot Curve: Discount Factors",
            font=dict(family="Inter, system-ui, sans-serif", size=20, color="#101010"),
        ),
        xaxis=dict(
            title="Time to Maturity (years)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        yaxis=dict(
            title="Discount Factor",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            font=dict(family="Inter, system-ui, sans-serif", size=12, color="#101010"),
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#E5E7EB",
            borderwidth=1,
        ),
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            font_family="Inter, system-ui, sans-serif",
            font_color="#101010",
            bordercolor="#E5E7EB",
        ),
        hovermode="x unified",
        font=dict(family="Inter, system-ui, sans-serif"),
    )

    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(
        go.Scatter(
            x=spot_curve["ttm"],
            y=spot_curve["r_semi"] * 100,
            mode="lines+markers",
            name="Semiannual Rate",
            line=dict(color="#3B82F6", width=2.5),
            marker=dict(size=6, color="#3B82F6"),
            hovertemplate="T: %{x:.4f}<br>Rate: %{y:.4f}%<extra></extra>",
        )
    )

    fig2.add_trace(
        go.Scatter(
            x=spot_curve["ttm"],
            y=spot_curve["r_cont"] * 100,
            mode="lines+markers",
            name="Continuous Rate",
            line=dict(color="#10B981", width=2.5),
            marker=dict(size=6, color="#10B981"),
            hovertemplate="T: %{x:.4f}<br>Rate: %{y:.4f}%<extra></extra>",
        )
    )

    fig2.update_layout(
        title=dict(
            text="Spot Curve: Interest Rates",
            font=dict(family="Inter, system-ui, sans-serif", size=20, color="#101010"),
        ),
        xaxis=dict(
            title="Time to Maturity (years)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        yaxis=dict(
            title="Rate (%)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            font=dict(family="Inter, system-ui, sans-serif", size=12, color="#101010"),
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#E5E7EB",
            borderwidth=1,
        ),
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            font_family="Inter, system-ui, sans-serif",
            font_color="#101010",
            bordercolor="#E5E7EB",
        ),
        hovermode="x unified",
        font=dict(family="Inter, system-ui, sans-serif"),
    )

    fig2.show()

In [171]:
def plot_forward_rates(forward_rates: pl.DataFrame):
    midpoint = (forward_rates["T_start"] + forward_rates["T_end"]) / 2

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=midpoint,
            y=forward_rates["forward_discount"],
            mode="lines+markers",
            name="Forward Discount f(T,T+0.5)",
            line=dict(color="#F59E0B", width=2.5),
            marker=dict(size=6, color="#F59E0B"),
            customdata=forward_rates.select(["T_start", "T_end"]).to_numpy(),
            hovertemplate="[%{customdata[0]:.4f}, %{customdata[1]:.4f}]<br>f: %{y:.4f}<extra></extra>",
        )
    )

    fig.update_layout(
        title=dict(
            text="6-Month Forward Discount Factors",
            font=dict(family="Inter, system-ui, sans-serif", size=20, color="#101010"),
        ),
        xaxis=dict(
            title="Interval Midpoint (years)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        yaxis=dict(
            title="Forward Discount Factor",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            font=dict(family="Inter, system-ui, sans-serif", size=12, color="#101010"),
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#E5E7EB",
            borderwidth=1,
        ),
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            font_family="Inter, system-ui, sans-serif",
            font_color="#101010",
            bordercolor="#E5E7EB",
        ),
        font=dict(family="Inter, system-ui, sans-serif"),
    )

    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(
        go.Scatter(
            x=midpoint,
            y=forward_rates["forward_rate_semi"] * 100,
            mode="lines+markers",
            name="Semiannual Forward Rate",
            line=dict(color="#EC4899", width=2.5),
            marker=dict(size=6, color="#EC4899"),
            customdata=forward_rates.select(["T_start", "T_end"]).to_numpy(),
            hovertemplate="[%{customdata[0]:.4f}, %{customdata[1]:.4f}]<br>Rate: %{y:.4f}%<extra></extra>",
        )
    )

    fig2.add_trace(
        go.Scatter(
            x=midpoint,
            y=forward_rates["forward_rate_cont"] * 100,
            mode="lines+markers",
            name="Continuous Forward Rate",
            line=dict(color="#06B6D4", width=2.5),
            marker=dict(size=6, color="#06B6D4"),
            customdata=forward_rates.select(["T_start", "T_end"]).to_numpy(),
            hovertemplate="[%{customdata[0]:.4f}, %{customdata[1]:.4f}]<br>Rate: %{y:.4f}%<extra></extra>",
        )
    )

    fig2.update_layout(
        title=dict(
            text="6-Month Forward Rates",
            font=dict(family="Inter, system-ui, sans-serif", size=20, color="#101010"),
        ),
        xaxis=dict(
            title="Interval Midpoint (years)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        yaxis=dict(
            title="Rate (%)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            font=dict(family="Inter, system-ui, sans-serif", size=12, color="#101010"),
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#E5E7EB",
            borderwidth=1,
        ),
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            font_family="Inter, system-ui, sans-serif",
            font_color="#101010",
            bordercolor="#E5E7EB",
        ),
        hovermode="x unified",
        font=dict(family="Inter, system-ui, sans-serif"),
    )

    fig2.show()

In [172]:
def plot_spot_and_forward_curves(spot_curve: pl.DataFrame, forward_rates: pl.DataFrame):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=spot_curve["ttm"],
            y=spot_curve["r_cont"] * 100,
            mode="lines+markers",
            name="Spot Rate",
            line=dict(color="#3B82F6", width=2.5),
            marker=dict(size=6, color="#3B82F6"),
            hovertemplate="T: %{x:.4f}<br>Spot: %{y:.4f}%<extra></extra>",
        )
    )

    midpoint = (forward_rates["T_start"] + forward_rates["T_end"]) / 2
    fig.add_trace(
        go.Scatter(
            x=midpoint,
            y=forward_rates["forward_rate_cont"] * 100,
            mode="lines+markers",
            name="6-Month Forward Rate",
            line=dict(color="#10B981", width=2.5),
            marker=dict(size=6, color="#10B981"),
            customdata=forward_rates.select(["T_start", "T_end"]).to_numpy(),
            hovertemplate="[%{customdata[0]:.4f}, %{customdata[1]:.4f}]<br>Forward: %{y:.4f}%<extra></extra>",
        )
    )

    fig.update_layout(
        title=dict(
            text="Spot Rate vs Forward Rate Curves",
            font=dict(family="Inter, system-ui, sans-serif", size=20, color="#101010"),
        ),
        xaxis=dict(
            title="Time to Maturity (years)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        yaxis=dict(
            title="Rate (%)",
            gridcolor="#E5E7EB",
            title_font=dict(
                family="Inter, system-ui, sans-serif", size=14, color="#101010"
            ),
            tickfont=dict(color="#101010"),
        ),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            font=dict(family="Inter, system-ui, sans-serif", size=12, color="#101010"),
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#E5E7EB",
            borderwidth=1,
        ),
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            font_family="Inter, system-ui, sans-serif",
            font_color="#101010",
            bordercolor="#E5E7EB",
        ),
        font=dict(family="Inter, system-ui, sans-serif"),
    )

    fig.show()


# Bootstrap the Spot Curve

In [173]:
data = get_data(
    source="data/treasury_quotes_2025-11-28.xlsx", sheet_name="selected quotes"
)

spot_curve = calculate_spot_curve(data)
forward_curve = calculate_forward_curve(spot_curve)
t5_results = spot_curve.filter(pl.col("ttm").round(2) == 5.00)

In [174]:
plot_spot_curve(spot_curve)

In [175]:
print(
    f"""Maturity T = 5 years
    Discount Factor:\t{t5_results["discount_factor"][0]:.4f}
    Semiannual Rate:\t{t5_results["r_semi"][0]:.4f}
    Continuous Rate:\t{t5_results["r_cont"][0]:.4f}
"""
)

Maturity T = 5 years
    Discount Factor:	0.8365
    Semiannual Rate:	0.0360
    Continuous Rate:	0.0357



# Forward Discount Factors

In [176]:
plot_forward_rates(forward_curve)

The forward discount factor from time $T_1$ to $T_2$ is:

$$f(T_1,T_2)= \frac{Z(T_2)}{Z(T_1)}​$$

The forward rates are then calculated as:
- Semiannual: $r_{\text{fwd,semi}} = 2 \left[ \left( \frac{1}{f} \right)^{\frac{1}{2\Delta T}} - 1 \right]$
- Continuous: $r_{\text{fwd,cont}} = -\frac{\ln(f)}{\Delta T}$

In [177]:
t5_forward = forward_curve.filter(
    (pl.col("T_start") >= 4.9) & (pl.col("T_start") <= 5.1)
).with_columns(
    [
        pl.col("forward_discount").round(4),
        pl.col("forward_rate_semi").round(4),
        pl.col("forward_rate_cont").round(4),
    ]
)

print("Forward rates from T=5 to T=5.5:")
print(t5_forward)

Forward rates from T=5 to T=5.5:
shape: (1, 5)
┌──────────┬─────────┬──────────────────┬───────────────────┬───────────────────┐
│ T_start  ┆ T_end   ┆ forward_discount ┆ forward_rate_semi ┆ forward_rate_cont │
│ ---      ┆ ---     ┆ ---              ┆ ---               ┆ ---               │
│ f64      ┆ f64     ┆ f64              ┆ f64               ┆ f64               │
╞══════════╪═════════╪══════════════════╪═══════════════════╪═══════════════════╡
│ 5.004791 ┆ 5.50308 ┆ 0.9787           ┆ 0.0437            ┆ 0.0432            │
└──────────┴─────────┴──────────────────┴───────────────────┴───────────────────┘


# Forward Curve Analysis

In [178]:
plot_spot_and_forward_curves(spot_curve, forward_curve)

In [179]:
max_forward = forward_curve.sort("forward_rate_cont", descending=True).head(1)

print(f"""Highest forward rate:
    Interval: T={max_forward["T_start"][0]:.4f} to T={max_forward["T_end"][0]:.4f}
    Forward rate (cont): {max_forward["forward_rate_cont"][0] * 100:.4f}%
    Forward rate (semi): {max_forward["forward_rate_semi"][0] * 100:.4f}%""")

Highest forward rate:
    Interval: T=7.0062 to T=7.4606
    Forward rate (cont): 4.9667%
    Forward rate (semi): 5.0289%


When the forward rate exceeds the spot rate at a given maturity, the spot curve is upward sloping in that region. This follows from the relationship and vice-versa:
$$f(T, T+\Delta) = r(T) + \frac{\partial r}{\partial T} \cdot \Delta T + O(\Delta T^2)$$
So if $f(T, T+\Delta) > r(T)$ , then $\frac{\partial r}{\partial T} > 0$ (positive slope, vice-versa).